# Deepface를 이용해 정확도 높은 얼굴인식 프로그램 만들기
https://github.com/serengil/deepface

In [1]:
import gradio as gr
from deepface import DeepFace
import cv2
import numpy as np

/home/haram/miniforge3/envs/deepface/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-12-08 09:36:52.280723: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-08 09:36:52.296253: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-08 09:36:52.760879: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow wit

# 유사도를 %로 표시하는 함수

In [2]:
def convert_distance_to_similarity(distance, threshold, verified):
    if verified:
        # 일치할 경우
        ratio = max(0, 1 -(distance/threshold))
        return round(90 + ratio * 10, 2)
    else:
        # 불일치 할 경우
        ratio = max(0, 1 - (distance/(threshold * 2)))
        return round(ratio * 60, 2)
    

# 얼굴 유사도 비교 함수 (ArcFace)

In [3]:
def image_to_text(image1, image2):
    # 이미지가 없는 경우 예외처리
    if image1 is None or image2 is None:
        return "이미지를 두 장 모두 업로드 하세요."
    
    img1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    img2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    
    cv2.imwrite("face1.jpg", img1)
    cv2.imwrite("face2.jpg", img2)
    
    try:
        threshold = 0.68 # ArcFace 기준 두 이미지가 일치하는 기준
        result = DeepFace.verify('face1.jpg', 'face2.jpg', model_name="ArcFace", enforce_detection=True)
        distance = result['distance']
        verified = result['verified']
        similarity_score = convert_distance_to_similarity(distance, threshold, verified)
        result_text = f"ArcFace 기준 얼굴 유사도 분석:\n유사도: {similarity_score:.2f}%\n얼굴 일치여부: {'일치' if verified else '불일치'}"
        return result_text
    except Exception as e:
        return f"오류: {str(e)}"
    

In [4]:
with gr.Blocks() as app:
    gr.Markdown("얼굴 유사도 비교")
    with gr.Tab('image Upload'):
        with gr.Row():
            with gr.Column():
                image1 = gr.Image(label="first_image")
            with gr.Column():
                image2 = gr.Image(label="second_image")
    output = gr.Textbox(label="얼굴 유사도: ", lines=5)
    convert_btn = gr.Button("유사도 비교하기")
    convert_btn.click(
        fn=image_to_text, inputs=[image1, image2], outputs=output)
app.launch(inline=False, share=False)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [5]:
app.close()

Closing server running on port: 7860


# 여러 모델과 유사도 측정 측도를 변경하면서 유사도 비교하기

모델목록

In [6]:
models = ['ArcFace', 'Facenet512', 'SFace', 'Facenet', 'GhostFaceNet', 'OpenFace','DeepID']

모델별 추천 metric

In [7]:
recommended_metric = dict(ArcFace='cosine', Facenet512='euclidean_l2', SFace='cosine', 
                          Facenet='euclidean_l2', GhostFaceNet='cosine',
                          OpenFace='euclidean_l2', DeepID='euclidean')
recommended_metric

{'ArcFace': 'cosine',
 'Facenet512': 'euclidean_l2',
 'SFace': 'cosine',
 'Facenet': 'euclidean_l2',
 'GhostFaceNet': 'cosine',
 'OpenFace': 'euclidean_l2',
 'DeepID': 'euclidean'}

모델별 threshold

In [8]:
model_threshold = dict(ArcFace=0.68, Facenet512=0.3, SFace=0.593, 
                          Facenet=0.4, GhostFaceNet=0.25,
                          OpenFace=0.55, DeepID=0.17)
model_threshold 

{'ArcFace': 0.68,
 'Facenet512': 0.3,
 'SFace': 0.593,
 'Facenet': 0.4,
 'GhostFaceNet': 0.25,
 'OpenFace': 0.55,
 'DeepID': 0.17}

In [9]:
def image_to_text2(image1, image2, model):
    # 이미지가 없는 경우 예외처리
    if image1 is None or image2 is None:
        return "이미지를 두 장 모두 업로드 하세요."
    
    img1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    img2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    
    # 추천 metric / threshold
    metric = recommended_metric.get(model, 'cosine')
    threshold = model_threshold.get(model, 0.5)
    
    try:
        result = DeepFace.verify(img1, img2, model_name=model,
                                 distance_metric=metric,
                                 detector_backend="opencv",
                                 enforce_detection=True)
        distance = result['distance']
        verified = result['verified']
        similarity_score = convert_distance_to_similarity(distance, threshold, verified)
        result_text = (f"ArcFace 기준 얼굴 유사도 분석:\n"
                       f"거리({metric}): {distance:.4f}\n"
                       f"유사도: {similarity_score:.2f}%\n"
                       f"얼굴 일치여부: {'일치' if verified else '불일치'}")
        return result_text
    except Exception as e:
        return f"오류: {str(e)}"

In [10]:
with gr.Blocks() as app:
    gr.Markdown("얼굴 유사도 비교")
    with gr.Tab('image Upload'):
        with gr.Row():
            with gr.Column():
                image1 = gr.Image(label="first_image")
            with gr.Column():
                image2 = gr.Image(label="second_image")
        model = gr.Dropdown(
            label="모델선택",
            choices=models,
            value="ArcFace")
        
    output = gr.Textbox(label="얼굴 유사도: ", lines=10)
    convert_btn = gr.Button("유사도 비교하기")
    convert_btn.click(
        fn=image_to_text2, inputs=[image1, image2, model], outputs=output)
app.launch(inline=False, share=False)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [11]:
app.close()

Closing server running on port: 7860
